In [3]:
pip install requests
pip install beautifulsoup4


SyntaxError: invalid syntax (3223553644.py, line 1)

In [4]:
import requests
from bs4 import BeautifulSoup

url = (
    "https://www.coursera.org/search"
    "?productTypeDescription=Professional%20Certificates&sortBy=BEST_MATCH"
)

response = requests.get(url)
response.raise_for_status()  # stop if the request failed

soup = BeautifulSoup(response.text, 'html.parser')

for card in soup.select('li')[:10]:  # Adjust the selector if necessary
    # Title
    title_tag = card.select_one('h2') or card.select_one('a[href*="/learn/"]')
    title = title_tag.get_text(strip=True) if title_tag else None

    # Provider
    provider_tag = card.select_one('G')  # Might need to adjust to proper tag
    provider = provider_tag.get_text(strip=True) if provider_tag else None

    # Rating
    rating_tag = card.find(string=lambda text: text and "Rating" in text)
    rating = rating_tag.split(' ')[0] if rating_tag else None

    # Reviews
    reviews_tag = card.find(string=lambda text: text and "reviews" in text)
    reviews = reviews_tag.split(' ')[0] if reviews_tag else None

    # Level & Duration
    meta_text = card.get_text(separator='|')
    # naive parsing
    level = None
    duration = None
    parts = [part.strip() for part in meta_text.split('|')]
    for part in parts:
        if 'Beginner' in part or 'Advanced' in part:
            level = part
        if 'Months' in part or 'Week' in part:
            duration = part

    print({
        'title': title,
        'provider': provider,
        'rating': rating,
        'reviews': reviews,
        'level': level,
        'duration': duration,
    })


{'title': None, 'provider': None, 'rating': None, 'reviews': None, 'level': None, 'duration': None}
{'title': None, 'provider': None, 'rating': None, 'reviews': None, 'level': None, 'duration': None}
{'title': None, 'provider': None, 'rating': None, 'reviews': None, 'level': None, 'duration': None}
{'title': None, 'provider': None, 'rating': None, 'reviews': None, 'level': None, 'duration': None}
{'title': None, 'provider': None, 'rating': None, 'reviews': None, 'level': None, 'duration': None}
{'title': None, 'provider': None, 'rating': None, 'reviews': None, 'level': None, 'duration': None}
{'title': None, 'provider': None, 'rating': None, 'reviews': None, 'level': None, 'duration': None}
{'title': None, 'provider': None, 'rating': None, 'reviews': None, 'level': None, 'duration': None}
{'title': None, 'provider': None, 'rating': 'Rating,', 'reviews': '173K', 'level': 'Beginner Â· Professional Certificate Â· 3 - 6 Months', 'duration': 'Beginner Â· Professional Certificate Â· 3 - 6 Mo

In [6]:
import requests
from bs4 import BeautifulSoup

url = (
    "https://www.coursera.org/search"
    "?productTypeDescription=Professional%20Certificates&sortBy=BEST_MATCH"
)

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')

# Each course card is an <li> tag (based on your example)
course_cards = soup.select('li')

for card in course_cards:
    # Course Name
    course_name_tag = card.select_one('a.cds-CommonCard-titleLink h3')
    course_name = course_name_tag.text.strip() if course_name_tag else None

    # Provider
    provider_tag = card.select_one('p.cds-ProductCard-partnerNames')
    provider = provider_tag.text.strip() if provider_tag else None

    # Skills you'll gain
    skills_tag = card.find('p', string=lambda t: t and "Skills you'll gain" in t)
    if not skills_tag:
        # Alternative: find p containing strong tag with text "Skills you'll gain"
        p_tags = card.select('p')
        skills_tag = None
        for p in p_tags:
            strong_tag = p.find('strong')
            if strong_tag and "Skills you'll gain" in strong_tag.text:
                skills_tag = p
                break
    skills = None
    if skills_tag:
        # remove the label "Skills you'll gain: "
        skills = skills_tag.text.replace("Skills you'll gain:", "").strip()

    # Rating
    rating_div = card.find('div', attrs={'aria-label': 'Rating'})
    rating = None
    if rating_div:
        rating_span = rating_div.find('span')
        rating = rating_span.text.strip() if rating_span else None

    # Total Reviews
    # Looks like a div with class "css-vac8rf" containing text like "173K reviews"
    review_divs = card.select('div.css-vac8rf')
    total_reviews = None
    for div in review_divs:
        text = div.text.strip()
        if "reviews" in text:
            total_reviews = text
            break

    # For (level, certificate type, duration)
    metadata_p = card.select_one('div.cds-CommonCard-metadata p.css-vac8rf')
    metadata = metadata_p.text.strip() if metadata_p else None

    if course_name:  # print only if course found (avoid empty results)
        print({
            'Course Name': course_name,
            'Provider': provider,
            
            'Rating': rating,
            'Total Reviews': total_reviews,
            'Metadata': metadata
        })


{'Course Name': 'Google Data Analytics', 'Provider': 'Google', 'Rating': '4.8', 'Total Reviews': '173K reviews', 'Metadata': 'Beginner Â· Professional Certificate Â· 3 - 6 Months'}
{'Course Name': 'Google Cybersecurity', 'Provider': 'Google', 'Rating': '4.8', 'Total Reviews': '56K reviews', 'Metadata': 'Beginner Â· Professional Certificate Â· 3 - 6 Months'}
{'Course Name': 'Google Project Management:', 'Provider': 'Google', 'Rating': '4.8', 'Total Reviews': '133K reviews', 'Metadata': 'Beginner Â· Professional Certificate Â· 3 - 6 Months'}
{'Course Name': 'Google Digital Marketing & E-commerce', 'Provider': 'Google', 'Rating': '4.8', 'Total Reviews': '43K reviews', 'Metadata': 'Beginner Â· Professional Certificate Â· 3 - 6 Months'}
{'Course Name': 'Google IT Support', 'Provider': 'Google', 'Rating': '4.8', 'Total Reviews': '206K reviews', 'Metadata': 'Beginner Â· Professional Certificate Â· 3 - 6 Months'}
{'Course Name': 'Google UX Design', 'Provider': 'Google', 'Rating': '4.8', 'Total

In [12]:
import requests
from bs4 import BeautifulSoup

url = (
    "https://www.coursera.org/search"
    "?productTypeDescription=Professional%20Certificates&sortBy=BEST_MATCH"
)

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')

course_cards = soup.select('li')


for card in course_cards[:10]:
    # extract data...

# for card in course_cards:
    # Course Name
    course_name_tag = card.select_one('a.cds-CommonCard-titleLink h3')
    course_name = course_name_tag.text.strip() if course_name_tag else None

    # Provider
    provider_tag = card.select_one('p.cds-ProductCard-partnerNames')
    provider = provider_tag.text.strip() if provider_tag else None

    # Skills you'll gain
    skills_tag = None
    # First try direct string match
    skills_tag = card.find('p', string=lambda t: t and "Skills you'll gain" in t)
    if not skills_tag:
        p_tags = card.select('p')
        for p in p_tags:
            strong_tag = p.find('strong')
            if strong_tag and "Skills you'll gain" in strong_tag.text:
                skills_tag = p
                break
    skills = None
    if skills_tag:
        skills = skills_tag.text.replace("Skills you'll gain:", "").strip()

    # Rating
    rating_div = card.find('div', attrs={'aria-label': 'Rating'})
    rating = None
    if rating_div:
        rating_span = rating_div.find('span')
        rating = rating_span.text.strip() if rating_span else None

    # Total Reviews
    total_reviews = None
    review_divs = card.select('div.css-vac8rf')
    for div in review_divs:
        text = div.text.strip()
        if "reviews" in text:
            total_reviews = text
            break

    # Metadata: level and duration
    metadata_p = card.select_one('div.cds-CommonCard-metadata p.css-vac8rf')
    metadata = metadata_p.text.strip() if metadata_p else None

    level = None
    duration = None
    if metadata:
        parts = [part.strip() for part in metadata.split('·')]
        if len(parts) >= 1:
            level = parts[0]
        if len(parts) >= 3:
            duration = parts[2]

    if course_name:
        print({
            'Course Name': course_name,
            'Provider': provider,
           
            'Rating': rating,
            'Total Reviews': total_reviews,
            'Level': level,
            'Duration': duration
        })


{'Course Name': 'Google Data Analytics', 'Provider': 'Google', 'Rating': '4.8', 'Total Reviews': '173K reviews', 'Level': 'Beginner Â', 'Duration': '3 - 6 Months'}
{'Course Name': 'Google Cybersecurity', 'Provider': 'Google', 'Rating': '4.8', 'Total Reviews': '56K reviews', 'Level': 'Beginner Â', 'Duration': '3 - 6 Months'}


In [14]:
import requests
from bs4 import BeautifulSoup

url = (
    "https://www.coursera.org/search"
    "?productTypeDescription=Professional%20Certificates&sortBy=BEST_MATCH"
)

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')

course_cards = soup.select('li')

num_courses = 100  # Change this to scrape more or fewer courses

for card in course_cards[:num_courses]:
    # Course Name
    course_name_tag = card.select_one('a.cds-CommonCard-titleLink h3')
    course_name = course_name_tag.text.strip() if course_name_tag else None

    # Provider
    provider_tag = card.select_one('p.cds-ProductCard-partnerNames')
    provider = provider_tag.text.strip() if provider_tag else None

    # Skills you'll gain
    skills_tag = None
    skills_tag = card.find('p', string=lambda t: t and "Skills you'll gain" in t)
    if not skills_tag:
        p_tags = card.select('p')
        for p in p_tags:
            strong_tag = p.find('strong')
            if strong_tag and "Skills you'll gain" in strong_tag.text:
                skills_tag = p
                break
    skills = None
    if skills_tag:
        skills = skills_tag.text.replace("Skills you'll gain:", "").strip()

    # Rating
    rating_div = card.find('div', attrs={'aria-label': 'Rating'})
    rating = None
    if rating_div:
        rating_span = rating_div.find('span')
        rating = rating_span.text.strip() if rating_span else None

    # Total Reviews
    total_reviews = None
    review_divs = card.select('div.css-vac8rf')
    for div in review_divs:
        text = div.text.strip()
        if "reviews" in text:
            total_reviews = text
            break

    # Metadata: level and duration
    metadata_p = card.select_one('div.cds-CommonCard-metadata p.css-vac8rf')
    metadata = metadata_p.text.strip() if metadata_p else None

    level = None
    duration = None
    if metadata:
        parts = [part.strip() for part in metadata.split('·')]
        if len(parts) >= 1:
            level = parts[0]
        if len(parts) >= 3:
            duration = parts[2]

    if course_name:
        print({
            'Course Name': course_name,
            'Provider': provider,
            'Skills': skills,
            'Rating': rating,
            'Total Reviews': total_reviews,
            'Level': level,
            'Duration': duration
        })


{'Course Name': 'Google Data Analytics', 'Provider': 'Google', 'Skills': 'Data Storytelling, Rmarkdown, Data Literacy, Data Visualization, Data Presentation, Data Ethics, Data Cleansing, Interactive Data Visualization, Data Validation, Ggplot2, Tableau Software, Sampling (Statistics), Presentations, Spreadsheet Software, Data Analysis, Data Visualization Software, Stakeholder Communications, Data Processing, Interviewing Skills, Applicant Tracking Systems', 'Rating': '4.8', 'Total Reviews': '173K reviews', 'Level': 'Beginner Â', 'Duration': '3 - 6 Months'}
{'Course Name': 'Google Cybersecurity', 'Provider': 'Google', 'Skills': 'Threat Modeling, Network Security, Incident Response, Vulnerability Management, Computer Security Incident Management, Hardening, Intrusion Detection and Prevention, Cyber Threat Intelligence, Threat Management, Cyber Attacks, Cybersecurity, Network Protocols, Cloud Security, Vulnerability Assessments, Bash (Scripting Language), Debugging, Linux, Interviewing Sk

In [15]:
import requests
from bs4 import BeautifulSoup

url = (
    "https://www.coursera.org/search"
    "?productTypeDescription=Professional%20Certificates&sortBy=BEST_MATCH"
)

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')

course_cards = soup.select('li.cds-ProductCard')
print(f"Found {len(course_cards)} course cards.")

for card in course_cards[:10]:
    title_tag = card.select_one('a.cds-CommonCard-titleLink h3')
    title = title_tag.text.strip() if title_tag else 'No title'
    print(title)


Found 0 course cards.


In [16]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def get_partners():
    url = "https://www.coursera.org/api/partners.v1"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    partner_map = {p['id']: p['name'] for p in data.get('elements', [])}
    return partner_map

def get_certificates(limit=10):
    url = "https://www.coursera.org/api/professional-certificates.v1"
    params = {
        'q': 'search',
        'query': 'professional certificates',
        'limit': limit,
        'fields': 'name,slug,partnerIds,description'
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json().get('elements', [])

def scrape_skills_from_page(slug):
    """Scrape skills from the certificate’s Coursera page."""
    base_url = f"https://www.coursera.org/professional-certificates/{slug}"
    response = requests.get(base_url)
    if response.status_code != 200:
        return None
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Look for the skills section
    # Inspect page to find where skills are listed; e.g., <p> with "Skills you'll gain"
    skills_tag = None
    for p in soup.find_all('p'):
        if p.strong and "Skills you'll gain" in p.strong.text:
            skills_tag = p
            break
    
    if skills_tag:
        # Remove the "Skills you'll gain:" label and clean text
        skills_text = skills_tag.get_text(separator=', ', strip=True)
        skills_text = skills_text.replace("Skills you'll gain:", "").strip()
        return skills_text
    return None

def main():
    partner_map = get_partners()
    certificates = get_certificates(limit=10)
    
    with open('coursera_certificates.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Slug', 'Providers', 'Description', 'Skills'])
        
        for cert in certificates:
            name = cert.get('name')
            slug = cert.get('slug')
            partners = [partner_map.get(pid, "Unknown") for pid in cert.get('partnerIds', [])]
            description_html = cert.get('description', '')
            description_text = BeautifulSoup(description_html, 'html.parser').get_text(separator=' ', strip=True)
            
            skills = scrape_skills_from_page(slug)
            if skills is None:
                skills = "Not found"
            
            print(f"Saving: {name} | Skills: {skills[:50]}...")
            
            writer.writerow([name, slug, ", ".join(partners), description_text, skills])
            
            time.sleep(2)  # polite delay to avoid overloading server

if __name__ == "__main__":
    main()


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import requests
from bs4 import BeautifulSoup

# Coursera API URL for Professional Certificates search
api_url = "https://www.coursera.org/api/professional-certificates.v1"

params = {
    'q': 'search',
    'query': 'professional certificates',
    'limit': 10,
    'fields': 'name,partnerIds,description,slug'
}

response = requests.get(api_url, params=params)
response.raise_for_status()
data = response.json()

# Get partner info (to map partner IDs to partner names)
partners_url = "https://www.coursera.org/api/partners.v1"
partners_response = requests.get(partners_url)
partners_response.raise_for_status()
partners_data = partners_response.json()
partner_map = {p['id']: p['name'] for p in partners_data.get('elements', [])}

for course in data.get('elements', []):
    course_name = course.get('name')
    partner_ids = course.get('partnerIds', [])
    providers = [partner_map.get(pid, "Unknown") for pid in partner_ids]
    
    description_html = course.get('description', '')
    # Parse description HTML with BeautifulSoup if needed (for example, to clean text)
    soup = BeautifulSoup(description_html, 'html.parser')
    description_text = soup.get_text(separator=' ', strip=True)
    
    # Since the API doesn't give rating, total reviews, skills, or metadata, set to None or fetch from page if needed
    rating = None
    total_reviews = None
    skills = None
    level = None
    duration = None
    
    print({
        'Course Name': course_name,
        'Provider': ", ".join(providers),
        'Description': description_text,
        'Skills': skills,
        'Rating': rating,
        'Total Reviews': total_reviews,
        'Level': level,
        'Duration': duration
    })


Status code: 200
<html>
	<head>
		<title>Coursera - API Route Does Not Exist</title>
	</head>
	<body style="background-color:#e4e4e4">
		<div style="position:absolute; top:0; bottom:0; left:0; right:0; margin:auto; height:200px; width: 600px">
			<div style="text-align:center">
				<img src="https://s3.amazonaws.com/coursera/error_pages/coursera-logo.svg" width="400">
			</div>
			<h1 style="text-align:center; font-family:Helvetica, Arial, sans-serif; font-weight:100; color: #555">
				API Route Does Not Exist
	
Error decoding JSON: Expecting value: line 1 column 1 (char 0)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [32]:
import requests
from bs4 import BeautifulSoup
import csv

url = (
    "https://www.coursera.org/search"
    "?productTypeDescription=Professional%20Certificates&sortBy=BEST_MATCH"
)

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')

course_cards = soup.select('li')

courses_data = []

for card in course_cards:
    # Course Name
    course_name_tag = card.select_one('a.cds-CommonCard-titleLink h3')
    course_name = course_name_tag.text.strip() if course_name_tag else None

    # Provider
    provider_tag = card.select_one('p.cds-ProductCard-partnerNames')
    provider = provider_tag.text.strip() if provider_tag else None

    # Skills you'll gain
    skills_tag = None
    skills_tag = card.find('p', string=lambda t: t and "Skills you'll gain" in t)
    if not skills_tag:
        p_tags = card.select('p')
        for p in p_tags:
            strong_tag = p.find('strong')
            if strong_tag and "Skills you'll gain" in strong_tag.text:
                skills_tag = p
                break
    skills = None
    if skills_tag:
        skills = skills_tag.text.replace("Skills you'll gain:", "").strip()

    # Rating
    rating_div = card.find('div', attrs={'aria-label': 'Rating'})
    rating = None
    if rating_div:
        rating_span = rating_div.find('span')
        rating = rating_span.text.strip() if rating_span else None

    # Total Reviews
    total_reviews = None
    review_divs = card.select('div.css-vac8rf')
    for div in review_divs:
        text = div.text.strip()
        if "reviews" in text:
            total_reviews = text
            break

    # Metadata: level and duration
    metadata_p = card.select_one('div.cds-CommonCard-metadata p.css-vac8rf')
    metadata = metadata_p.text.strip() if metadata_p else None

    level = None
    duration = None
    if metadata:
        parts = [part.strip() for part in metadata.split('·')]
        if len(parts) >= 1:
            level = parts[0]
        if len(parts) >= 3:
            duration = parts[2]

    if course_name:
        courses_data.append({
            'Course Name': course_name,
            'Provider': provider,
            'Skills': skills,
            'Rating': rating,
            'Total Reviews': total_reviews,
            'Level': level,
            'Duration': duration
        })

# Save to CSV
csv_file = 'coursera_courses.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Course Name', 'Provider', 'Skills', 'Rating', 'Total Reviews', 'Level', 'Duration'])
    writer.writeheader()
    writer.writerows(courses_data)

print(f"Data saved to {csv_file}")


Data saved to coursera_courses.csv


In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# Setup Chrome options
options = Options()
# options.add_argument("--headless")  # Uncomment to run headless

driver = webdriver.Chrome(options=options)

url = "https://www.coursera.org/search?productTypeDescription=Professional%20Certificates&sortBy=BEST_MATCH"
driver.get(url)

# Scroll down multiple times to load more results
SCROLL_PAUSE_TIME = 3
last_height = driver.execute_script("return document.body.scrollHeight")

for _ in range(10):  # Scroll 10 times, adjust if needed
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Wait for course cards to be present
wait = WebDriverWait(driver, 15)
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.cds-CommonCard')))

course_cards = driver.find_elements(By.CSS_SELECTOR, 'li.cds-CommonCard')
print(f"Found {len(course_cards)} course cards")

data = []

for card in course_cards[:20]:  # Limit to first 20 courses, change as needed
    try:
        course_name = card.find_element(By.CSS_SELECTOR, 'a.cds-CommonCard-titleLink h3').text.strip()
    except:
        course_name = None

    try:
        provider = card.find_element(By.CSS_SELECTOR, 'p.cds-ProductCard-partnerNames').text.strip()
    except:
        provider = None

    try:
        rating = card.find_element(By.CSS_SELECTOR, 'div[aria-label^="Rating"] span').text.strip()
    except:
        rating = None

    try:
        total_reviews = None
        divs = card.find_elements(By.CSS_SELECTOR, 'div.css-vac8rf')
        for div in divs:
            text = div.text.strip()
            if "reviews" in text:
                total_reviews = text
                break
    except:
        total_reviews = None

    try:
        metadata_text = card.find_element(By.CSS_SELECTOR, 'div.cds-CommonCard-metadata p.css-vac8rf').text.strip()
        parts = [part.strip() for part in metadata_text.split('·')]
        level = parts[0] if len(parts) > 0 else None
        duration = parts[2] if len(parts) > 2 else None
    except:
        level = None
        duration = None

    data.append({
        'Course Name': course_name,
        'Provider': provider,
        'Rating': rating,
        'Total Reviews': total_reviews,
        'Level': level,
        'Duration': duration
    })

driver.quit()

# Save data to CSV
keys = data[0].keys() if data else []
with open('coursera_professional_certificates.csv', 'w', newline='', encoding='utf-8') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data)

print("Data saved to coursera_professional_certificates.csv")


TimeoutException: Message: 


In [28]:
import requests
import csv

url = "https://www.coursera.org/api/catalogResults.v2?q=search&query=professional%20certificate&limit=50&includes=partnerIds,courseId&fields=courseId,domainTypes,partners.v1(name),description"

headers = {
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)
data = response.json()

results = data["linked"]["courses"]

courses = []

for course in results:
    courses.append({
        "Course Name": course.get("name"),
        "Description": course.get("description"),
        "Course ID": course.get("id")
    })

# Save to CSV
with open("coursera_api_courses.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=courses[0].keys())
    writer.writeheader()
    writer.writerows(courses)

print("Saved to coursera_api_courses.csv")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [29]:
import requests
from bs4 import BeautifulSoup
import csv

# url = (
#     "https://www.coursera.org/search"
#     "?productTypeDescription=Professional%20Certificates&sortBy=BEST_MATCH"
# )

url=(
    "/home/student/Downloads/Courses.html"
)
response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')

course_cards = soup.select('li')

courses_data = []

for card in course_cards:
    # Course Name
    course_name_tag = card.select_one('a.cds-CommonCard-titleLink h3')
    course_name = course_name_tag.text.strip() if course_name_tag else None

    # Provider
    provider_tag = card.select_one('p.cds-ProductCard-partnerNames')
    provider = provider_tag.text.strip() if provider_tag else None

    # Skills you'll gain
    skills_tag = None
    skills_tag = card.find('p', string=lambda t: t and "Skills you'll gain" in t)
    if not skills_tag:
        p_tags = card.select('p')
        for p in p_tags:
            strong_tag = p.find('strong')
            if strong_tag and "Skills you'll gain" in strong_tag.text:
                skills_tag = p
                break
    skills = None
    if skills_tag:
        skills = skills_tag.text.replace("Skills you'll gain:", "").strip()

    # Rating
    rating_div = card.find('div', attrs={'aria-label': 'Rating'})
    rating = None
    if rating_div:
        rating_span = rating_div.find('span')
        rating = rating_span.text.strip() if rating_span else None

    # Total Reviews
    total_reviews = None
    review_divs = card.select('div.css-vac8rf')
    for div in review_divs:
        text = div.text.strip()
        if "reviews" in text:
            total_reviews = text
            break

    # Metadata: level and duration
    metadata_p = card.select_one('div.cds-CommonCard-metadata p.css-vac8rf')
    metadata = metadata_p.text.strip() if metadata_p else None

    level = None
    duration = None
    if metadata:
        parts = [part.strip() for part in metadata.split('·')]
        if len(parts) >= 1:
            level = parts[0]
        if len(parts) >= 3:
            duration = parts[2]

    if course_name:
        courses_data.append({
            'Course Name': course_name,
            'Provider': provider,
            'Skills': skills,
            'Rating': rating,
            'Total Reviews': total_reviews,
            'Level': level,
            'Duration': duration
        })

# Save to CSV
csv_file = 'coursera_courses.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Course Name', 'Provider', 'Skills', 'Rating', 'Total Reviews', 'Level', 'Duration'])
    writer.writeheader()
    writer.writerows(courses_data)

print(f"Data saved to {csv_file}")


MissingSchema: Invalid URL '/home/student/Downloads/Courses.html': No scheme supplied. Perhaps you meant https:///home/student/Downloads/Courses.html?

In [31]:
from bs4 import BeautifulSoup
import csv

# Path to your local HTML file
file_path = '/home/student/Downloads/Courses.html'

# Open and read the file
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Select all course cards - adjust this selector if needed based on your HTML
course_cards = soup.select('li')

courses_data = []

for card in course_cards:
    # Course Name
    course_name_tag = card.select_one('a.cds-CommonCard-titleLink h3')
    course_name = course_name_tag.text.strip() if course_name_tag else None

    # Provider
    provider_tag = card.select_one('p.cds-ProductCard-partnerNames')
    provider = provider_tag.text.strip() if provider_tag else None

    # Skills you'll gain
    skills_tag = card.find('p', string=lambda t: t and "Skills you'll gain" in t)
    if not skills_tag:
        for p in card.select('p'):
            strong_tag = p.find('strong')
            if strong_tag and "Skills you'll gain" in strong_tag.text:
                skills_tag = p
                break
    skills = skills_tag.text.replace("Skills you'll gain:", "").strip() if skills_tag else None

    # Rating
    rating_div = card.find('div', attrs={'aria-label': 'Rating'})
    rating = None
    if rating_div:
        rating_span = rating_div.find('span')
        rating = rating_span.text.strip() if rating_span else None

    # Total Reviews
    total_reviews = None
    review_divs = card.select('div.css-vac8rf')
    for div in review_divs:
        text = div.text.strip()
        if "reviews" in text:
            total_reviews = text
            break

    # Metadata (Level and Duration)
    metadata_p = card.select_one('div.cds-CommonCard-metadata p.css-vac8rf')
    level = None
    duration = None
    if metadata_p:
        parts = [part.strip() for part in metadata_p.text.split('·')]
        if len(parts) >= 1:
            level = parts[0]
        if len(parts) >= 3:
            duration = parts[2]

    if course_name:  # Only add if course name exists
        courses_data.append({
            'Course Name': course_name,
            'Provider': provider,
            'Skills': skills,
            'Rating': rating,
            'Total Reviews': total_reviews,
            'Level': level,
            'Duration': duration
        })

# Save to CSV
csv_file = 'coursera_courses_html.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Course Name', 'Provider', 'Skills', 'Rating', 'Total Reviews', 'Level', 'Duration'])
    writer.writeheader()
    writer.writerows(courses_data)

print(f"Data saved to {csv_file}")


Data saved to coursera_courses_html.csv


In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def get_courses_from_page(soup):
    course_cards = soup.select('li')
    page_courses = []

    for card in course_cards:
        # Course Name
        course_name_tag = card.select_one('a.cds-CommonCard-titleLink h3')
        course_name = course_name_tag.text.strip() if course_name_tag else None

        # Provider
        provider_tag = card.select_one('p.cds-ProductCard-partnerNames')
        provider = provider_tag.text.strip() if provider_tag else None

        # Skills you'll gain
        skills_tag = None
        skills_tag = card.find('p', string=lambda t: t and "Skills you'll gain" in t)
        if not skills_tag:
            p_tags = card.select('p')
            for p in p_tags:
                strong_tag = p.find('strong')
                if strong_tag and "Skills you'll gain" in strong_tag.text:
                    skills_tag = p
                    break
        skills = None
        if skills_tag:
            skills = skills_tag.text.replace("Skills you'll gain:", "").strip()

        # Rating
        rating_div = card.find('div', attrs={'aria-label': 'Rating'})
        rating = None
        if rating_div:
            rating_span = rating_div.find('span')
            rating = rating_span.text.strip() if rating_span else None

        # Total Reviews
        total_reviews = None
        review_divs = card.select('div.css-vac8rf')
        for div in review_divs:
            text = div.text.strip()
            if "reviews" in text:
                total_reviews = text
                break

        # Metadata: level and duration
        metadata_p = card.select_one('div.cds-CommonCard-metadata p.css-vac8rf')
        metadata = metadata_p.text.strip() if metadata_p else None

        level = None
        duration = None
        if metadata:
            parts = [part.strip() for part in metadata.split('·')]
            if len(parts) >= 1:
                level = parts[0]
            if len(parts) >= 3:
                duration = parts[2]

        if course_name:
            page_courses.append({
                'Course Name': course_name,
                'Provider': provider,
                'Skills': skills,
                'Rating': rating,
                'Total Reviews': total_reviews,
                'Level': level,
                'Duration': duration
            })

    return page_courses


# === Config ===
TARGET_COUNT = 100  # Change this to get more or fewer courses
BASE_URL = "https://www.coursera.org/search?productTypeDescription=Professional%20Certificates&sortBy=BEST_MATCH&page={}"

courses_data = []
page = 1

while len(courses_data) < TARGET_COUNT:
    print(f"Scraping page {page}...")

    url = BASE_URL.format(page)
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')
    page_courses = get_courses_from_page(soup)

    if not page_courses:
        print("No more courses found.")
        break

    courses_data.extend(page_courses)
    page += 1
    time.sleep(1)  # Be polite to the server

# Trim to exact target count
courses_data = courses_data[:TARGET_COUNT]

# Save to CSV
csv_file = 'coursera_courses_ajax.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Course Name', 'Provider', 'Skills', 'Rating', 'Total Reviews', 'Level', 'Duration'])
    writer.writeheader()
    writer.writerows(courses_data)

print(f"\n✅ Scraped {len(courses_data)} courses and saved to '{csv_file}'")


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...

✅ Scraped 100 courses and saved to 'coursera_courses_ajax.csv'
